In [1]:
import psycopg2
import nltk
import pandas as pd
import numpy as np
import re
import string
import spacy
import sklearn
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [2]:
# importing and creating df (has to have type_id)

# usage: specify file location, sample size and seed(used by random)
#filepath = '../Data_sample/FakeNewsCorpus_250.csv' # 250 rows of FakeNewsCorpus
filepath = '/Users/Master/Documents/KU/2.Semester/Datascience/1mioraw.csv'       # 1 mil rows raw
#s = 1000000                                            # desired sample size
#seed = 1                                           # seed used by Pseudorandom number generator

df = pd.read_csv(filepath, index_col = [0])
df["content"] = df["content"].astype(str)
# create type_id
df['type_id'] = df.groupby(['type']).ngroup()

df.head()

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,type_id
0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"Life is an illusion, at least on a quantum lev...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,NaN,[''],THE UNIVERSE ceases to exist when we are not l...,NaN,NaN,NaN,8
1,6,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,"Unfortunately, he hasn’t yet attacked her for ...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN,4
2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,The Los Angeles Police Department has been den...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN,4
3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,The White House has decided to quietly withdra...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",NaN,[''],NaN,NaN,NaN,NaN,4
4,9,barenakedislam.com,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,“The time has come to cut off the tongues of t...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"“Oh, Trump, you coward, you just wait, we will...","F.N. Lehner, Don Spilman, Clarence J. Feinour,...",NaN,[''],NaN,NaN,NaN,NaN,4


In [5]:
df.index

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       9924, 9925, 9926, 9927, 9928, 9929, 9930, 9931, 9932, 9933],
      dtype='object', length=1000000)

In [3]:
#1
df['id'] = pd.to_numeric(df['id'], errors = 'coerce', downcast = 'integer')
df.dropna(subset=['id'], inplace = True)
#df.drop_duplicates(subset = 'content', inplace = True)
df.drop_duplicates(subset = 'id', inplace = True)
df.id = df.id.astype(int)



In [4]:
df.loc[df['id'] == np.nan]

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,type_id


In [5]:
df.loc[df['id'].duplicated()]

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,type_id


In [6]:
df.reset_index(inplace = True, drop = True)
df.index.value_counts()

2047      1
983221    1
995507    1
997554    1
991409    1
         ..
717812    1
707571    1
705522    1
711665    1
0         1
Length: 999934, dtype: int64

In [36]:
#df.info()
#df.count

In [7]:
dupidx_lst = df.loc[df['content'].duplicated()].index.tolist()


In [8]:
df_Uniq = df.loc[~df.index.isin(dupidx_lst)]

In [11]:
df_Uniq

,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,type_id
0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"Life is an illusion, at least on a quantum lev...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,NaN,[''],THE UNIVERSE ceases to exist when we are not l...,NaN,NaN,NaN,8
1,6,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,"Unfortunately, he hasn’t yet attacked her for ...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN,4
2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,The Los Angeles Police Department has been den...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN,4
3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,The White House has decided to quietly withdra...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",NaN,[''],NaN,NaN,NaN,NaN,4
4,9,barenakedislam.com,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,“The time has come to cut off the tongues of t...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"“Oh, Trump, you coward, you just wait, we will...","F.N. Lehner, Don Spilman, Clarence J. Feinour,...",NaN,[''],NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999895,1170082,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1974ATO...,Raw content\n\nPAGE 01 NATO 05116 01 OF 02 201...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1974ATO05116,NaN,NaN,[''],NaN,View Tags,NaN,NaN,11
999906,1170093,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976ABU...,Raw content\n\nCONFIDENTIAL PAGE 01 ABU DH 000...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976ABUDH00021,NaN,NaN,[''],NaN,View Tags,NaN,NaN,11
999913,1170100,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976ANK...,Raw content\n\nCONFIDENTIAL PAGE 01 ANKARA 019...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976ANKARA01989,NaN,NaN,[''],NaN,View Tags,NaN,NaN,11
999923,1170110,wikileaks.org,unreliable,https://www.wikileaks.org/plusd/cables/1976ROM...,Raw content\n\nLIMITED OFFICIAL USE PAGE 01 RO...,2018-01-25 20:13:50.426130,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Cable: 1976ROME16372,NaN,NaN,[''],NaN,View Tags,NaN,NaN,11


In [12]:
df_Uniq['type'].loc[df_Uniq['type'] == 'reliable'].value_counts().min()

5222

In [13]:
df_Uniq.content.nunique()

660914

In [9]:
#USE THIS
# This can generate a dataset with random purmutation and a max size for each type(can be smaller if desired max is not possible)

# max size for type
max_size = 7000
# traning_set ratio - splits data into traning=ratio,  test and validate=(1-ratio)/2 ex. train=80%, test=10%, validate=10%
ratio=0.8
# Labels to include - ['fake', 'satire', 'bias', 'conspiracy', 'state', 'junksci', 'hate', 'clickbait', 'unreliable', 'political', 'reliable'] - all labels
use_types = ['fake', 'satire', 'bias', 'conspiracy', 'junksci', 'hate', 'clickbait', 'unreliable', 'political', 'reliable']
# Random seed
rnd = 1

# initialize dataframes
train    = pd.DataFrame(columns = df_Uniq.columns)
test     = pd.DataFrame(columns = df_Uniq.columns)

# add type to test splits
for t in use_types:

    # type size
    type_size = df_Uniq['type'].loc[df_Uniq['type'] == t].value_counts().min()

    # set size of type slice
    if type_size < max_size:
        tmp = df_Uniq.loc[df_Uniq['type'] == t].sample(n = type_size, random_state=rnd)
    else:
        tmp = df_Uniq.loc[df_Uniq['type'] == t].sample(n = max_size, random_state=rnd)

    # split current type
    train_tmp, test_tmp = np.split(tmp, [int(ratio * len(tmp))])

    # add tmp to dataframes
    train    = pd.concat([train, train_tmp])
    test     = pd.concat([test, test_tmp])
    
    # print split shape
    print("=>", t, tmp.shape, train_tmp.shape, test_tmp.shape)

print("\n[Final split]\ntrain, test==>", train.shape, test.shape)

=> fake (7000, 17) (5600, 17) (1400, 17)
=> satire (7000, 17) (5600, 17) (1400, 17)
=> bias (7000, 17) (5600, 17) (1400, 17)
=> conspiracy (7000, 17) (5600, 17) (1400, 17)
=> junksci (7000, 17) (5600, 17) (1400, 17)
=> hate (3112, 17) (2489, 17) (623, 17)
=> clickbait (7000, 17) (5600, 17) (1400, 17)
=> unreliable (7000, 17) (5600, 17) (1400, 17)
=> political (7000, 17) (5600, 17) (1400, 17)
=> reliable (5222, 17) (4177, 17) (1045, 17)

[Final split]
train, test==> (51466, 17) (12868, 17)


In [434]:
pd.merge(train,test, on =['content'], how ='inner')

,id_x,domain_x,type_x,url_x,content,scraped_at_x,inserted_at_x,updated_at_x,title_x,authors_x,...,updated_at_y,title_y,authors_y,keywords_y,meta_keywords_y,meta_description_y,tags_y,summary_y,source_y,type_id_y


In [87]:
#Just for faster calculation for details, dont run normally
# importing and creating df (has to have type_id)

# usage: specify file location, sample size and seed(used by random) 
#filepath = '/Users/Master/Documents/KU/2.Semester/Datascience/news_sample.csv' #
#s = 250                                            # desired sample size
#seed = 1                                           # seed used by Pseudorandom number generator

#df = pd.read_csv(filepath, index_col = [0])
#content = df['content']

In [10]:
import gensim 
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models.doc2vec import TaggedDocument, Doc2Vec 
from gensim.utils import tokenize


In [14]:
#train

In [11]:
#train.set_index('id', inplace = True)
from sklearn.utils import shuffle
train = shuffle(train, random_state = 1)
test = shuffle(test, random_state = 1)
train.reset_index(inplace = True, drop = True)
test.reset_index(inplace = True, drop = True)

In [128]:
#this is how the tokenizer works
list(tokenize("This , is 5to TOKEN", lowercase = True))

['this', 'is', 'to', 'token']

In [12]:
%%time
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
series_content = train.content
#contTok = series_content.apply(nltk.word_tokenize)
typeLst = train.type_id.tolist()
#np.unique(typeLst, return_counts = True)

CPU times: user 1.17 ms, sys: 20 µs, total: 1.19 ms
Wall time: 1.89 ms


In [13]:
test_content = test.content
test_typeLst = test.type_id.tolist()

In [549]:
#test_content
#test_typeLst

In [14]:
#Try gensim tokenizer
testTok = test_content.apply(tokenize,lowercase =True)
for i in testTok.index:
    testTok[i] = list(testTok[i])

In [15]:
#Try gensim tokenizer
contTok = series_content.apply(tokenize,lowercase =True)
for i in contTok.index:
    contTok[i] = list(contTok[i])

In [ ]:
#Maybe fallback tokenizer
#contTok1 = series_content.apply(nltk.word_tokenize)

In [422]:
train.type[0]

'junksci'

In [17]:
documents = [TaggedDocument(contTok, [i]) for i, contTok in enumerate(contTok)]
#contTok[291496]

In [28]:
#list(enumerate(contTok))[0]
#documents

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [21]:
model2 = Doc2Vec(documents, vector_size = 150, dm = 0, epochs=15, workers = 4)

In [32]:
%%time
model1 = Doc2Vec(documents, vector_size = , min_count = 3, window = 10, epochs=10, workers = 4)

CPU times: user 22min 15s, sys: 21 s, total: 22min 36s
Wall time: 10min 50s


In [ ]:
#Eventuel træn andre modeller herefter:
#%%time
#model2 = Doc2Vec(documents, vector_size = 200, dm = 1, min_count = 3, window = 10, epochs= 10, workers = 4)

In [22]:
model2.most_similar('apple')

[('introverted', 0.3632413148880005),
 ('johansen', 0.32919424772262573),
 ('trick', 0.31969472765922546),
 ('bulge', 0.30862855911254883),
 ('parents', 0.306522011756897),
 ('inr', 0.30332380533218384),
 ('reflective', 0.3025538921356201),
 ('ducing', 0.30231982469558716),
 ('prolifera', 0.30203330516815186),
 ('dialogues', 0.3010951280593872)]

In [23]:
model2.docvecs.count

51466

In [454]:
#documents[200]

In [24]:
model2.docvecs.most_similar(200)

[(47016, 0.7833322882652283),
 (39974, 0.7338676452636719),
 (41431, 0.7269080877304077),
 (2547, 0.7235113382339478),
 (9676, 0.7217190265655518),
 (39357, 0.7115262746810913),
 (45440, 0.7040430307388306),
 (18977, 0.6920127868652344),
 (30585, 0.6893031001091003),
 (5103, 0.6876428127288818)]

In [457]:
train.type.loc[train.type.index == 39974]

39974    fake
Name: type, dtype: object

In [319]:
#train.type.loc[train.index == 38424]

In [33]:
#See types of most similar documents for model2
#for idx in train.type.loc[train.type == 'fake'].index:
sim_labels1 = np.empty(10,dtype='object')
for i in range(len(sim_labels1)):
    sim_labels1[i] = train.type.loc[train.type.index == model2.docvecs.most_similar(51398)[i][0]].values[0]

np.unique(sim_labels1,return_counts=True)

(array(['clickbait', 'junksci', 'political'], dtype=object), array([3, 4, 3]))

In [29]:
#Lookup indeices for articles with certain types
train.type.loc[train.type == 'bias']

16       bias
21       bias
23       bias
34       bias
43       bias
         ... 
51398    bias
51399    bias
51400    bias
51416    bias
51464    bias
Name: type, Length: 5600, dtype: object

In [34]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import utils
from tqdm import tqdm
import multiprocessing


In [120]:
import numpy as np

In [121]:
#Model 1 traindata
train_arrays = np.zeros((model1.docvecs.count,150),dtype="float32")
train_labels = np.zeros(model1.docvecs.count,dtype="float32")
for i in range(model1.docvecs.count):
    train_arrays[i] = model1.docvecs[i]
    train_labels[i] = train.type_id[i]

In [477]:
model2.docvecs.count

51466

In [80]:
#Model 2 traindata:
train_arrays2 = np.zeros((model2.docvecs.count,150),dtype="float32")
train_labels2 = np.zeros(model2.docvecs.count,dtype="float32")
for i in range(model2.docvecs.count):
    train_arrays2[i] = model2.docvecs[i]
    train_labels2[i] = train.type_id[i]

In [37]:
#np.unique(train_labels, return_counts = True)

In [538]:
#train_arrays[0]
#train_labels[0]
#model1.docvecs.most_similar([train_arrays[0]])
#train.loc[train.index == 37400]

In [194]:
#How infer_vector works:
token = "this is new sentence".split()
new_vector = model1.infer_vector(token)
sims = model1.docvecs.most_similar([new_vector])
#model2.docvecs.most_similar([new_vec])

In [497]:
#new_vec = model2.infer_vector(contTok[2])

In [499]:
#model2.docvecs.most_similar([new_vec])

In [131]:
#Inferring vectors for testdata with model1
test_arrays = np.zeros((len(test),150),dtype = "float32")
for i in range(len(test)):
    test_arrays[i] = model1.infer_vector(testTok[i],steps = 15)   
test_labels = np.array(test_typeLst,dtype="float32")

#test_arrays = np.zeros((model1.docvecs.count,150),dtype="float32")
#test_labels = np.zeros(model1.docvecs.count,dtype="float32")

In [38]:
%%time
#Inferring vectors for testdata with model2
test_arrays2 = np.zeros((len(test),150),dtype = "float32")
for i in range(len(test)):
    test_arrays2[i] = model2.infer_vector(testTok[i], epochs = 20, alpha = 0.025)   
test_labels = np.array(test_typeLst,dtype="float32")

CPU times: user 6min 29s, sys: 5.3 s, total: 6min 34s
Wall time: 6min 48s


In [146]:
#testTok[300]

In [149]:
#test_labels[300]

In [584]:
#np.unique(test_labels, return_counts=True)

In [568]:
model1.docvecs.most_similar([test_arrays[200]])

[(32672, 0.5412675142288208),
 (10444, 0.5094727277755737),
 (23135, 0.49477219581604004),
 (5553, 0.4843190908432007),
 (33264, 0.4779159426689148),
 (32602, 0.4745362102985382),
 (460, 0.4699743688106537),
 (24024, 0.468875527381897),
 (11487, 0.4654442071914673),
 (2246, 0.46349072456359863)]

In [180]:
test.type.loc[test.type.index == 3642]

3642    clickbait
Name: type, dtype: object

In [39]:
#Find originalarticle types most similar to inferred vector by model1:
sim_labels = np.empty(10,dtype='object')
for i in range(len(sim_labels)):
    sim_labels[i] = train.type.loc[train.type.index == model1.docvecs.most_similar([test_arrays[10]])[i][0]].values[0]

np.unique(sim_labels,return_counts=True)

NameError: name 'model1' is not defined

In [350]:
model2.docvecs.most_similar([test_arrays2[200]])

[(15221, 0.460197776556015),
 (20051, 0.4442721903324127),
 (37955, 0.4413350224494934),
 (6278, 0.4317403733730316),
 (17709, 0.4302103519439697),
 (29647, 0.42933136224746704),
 (19457, 0.4270772933959961),
 (122, 0.42285194993019104),
 (32331, 0.42190662026405334),
 (21916, 0.421882688999176)]

In [49]:
train.type[10004]

'satire'

In [45]:
model2.docvecs.most_similar([test_arrays2[10]])

[(14004, 0.6460142731666565),
 (31820, 0.6411257982254028),
 (36654, 0.6238328218460083),
 (10004, 0.6183276176452637),
 (4258, 0.612575113773346),
 (9671, 0.6100050210952759),
 (45557, 0.6094460487365723),
 (36745, 0.6056312322616577),
 (46957, 0.6026122570037842),
 (12744, 0.6022600531578064)]

In [66]:
#Find originalarticle types most similar to inferred vector by model2:
sim_labels = np.empty(10,dtype='object')
for i in range(len(sim_labels)):
    sim_labels[i] = train.type.loc[train.type.index == model2.docvecs.most_similar([test_arrays2[75]])[i][0]].values[0]

np.unique(sim_labels,return_counts=True)

(array(['clickbait', 'political', 'reliable'], dtype=object), array([1, 2, 7]))

In [61]:
#Get indices for testarticles of a certain type:
test.type.loc[test.type == 'reliable'].index

Int64Index([   11,    17,    25,    29,    54,    63,    75,    88,    90,
              124,
            ...
            12742, 12748, 12762, 12784, 12803, 12816, 12818, 12826, 12851,
            12866],
           dtype='int64', length=1045)

In [184]:
#model1.docvecs.most_similar([test_arrays[10]])

In [185]:
#train.type.loc[train.type == 'bias'].index

In [86]:
from sklearn.preprocessing import StandardScaler
#standardizing data
scaler = StandardScaler()
scaler.fit(train_arrays2)
X_train = scaler.transform(train_arrays2)
X_test = scaler.transform(test_arrays2)

In [411]:
len(train.loc[train.type == 'hate'])

2489

In [405]:
4000/2489
2489/400


1.6070711128967456

In [527]:
#Now train a logistic classifier:
#can try gridsearch for parameters:
#Logistic = LogisticRegression()
#grid = {"C" :np.logspace(0,4,10), "penalty": ["l1", "l2"]}
#logClf = GridSearchCV(Logistic, grid, cv = 5)
#best_model = logClf.fit(X_train, train_labels)
clfLog = LogisticRegression(C = 0.01,multi_class = 'multinomial', solver = 'saga', penalty = 'l2', warm_start = True)
clfLog.fit(X_train, train_labels2)

#classifier.fit(X_train,train_labels)
#classifier.n_iter_

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=True)

In [528]:
clfLog.score(X_test, test_labels)

0.6565899906745415

In [414]:
y_true = test_labels
#y_true
y_pred = clfLog.predict(X_test)

In [415]:
confusion_matrix(y_true,y_pred)

array([[433,  59,  96,  13, 160,  49, 109,  45,  30,   6],
       [ 50, 546,  49,  12,  51,  42, 119,  51,  72,   8],
       [ 75,  60, 530,  14,  82,  63, 105,  22,  43,   6],
       [ 15,  18,  33, 852,  15,  33,  17,   5,  11,   1],
       [ 39,  38,  28,   8, 415,  11,  44,  21,  14,   5],
       [ 22,  41,  41,   7,  11, 807,  15,  21,  35,   0],
       [ 93, 137,  76,  14,  76,  22, 468,  58,  50,   6],
       [ 63,  64,  16,   5,  51,  25,  46, 692,  37,   1],
       [ 39,  79,  49,   8,  34,  31,  56,  18, 682,   4],
       [ 18,  40,  18,   4,  11,  11,  27,   9,   8, 854]])

In [287]:
accuracy_score(y_true,y_pred)

0.6406526031383144

In [530]:
#Try GaussianNB:
from sklearn.naive_bayes import GaussianNB
clfNB = GaussianNB()
clfNB.fit(X_train, train_labels2)

GaussianNB(priors=None, var_smoothing=1e-09)

In [531]:
clfNB.score(X_test,test_labels)

0.5231582219459123

In [252]:
from sklearn.svm import LinearSVC
LinSvm = LinearSVC()
LinSvm.fit(X_train, train_labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [253]:
LinSvm.score(X_test, test_labels)

0.6350410474903876

In [532]:
#rbf kernel = 0 .72, poly kernel = 0.7
from sklearn import svm
from sklearn.model_selection import GridSearchCV
#parameters = {'C':[1,10], 'kernel': ('linear','rbf')}
clf = svm.SVC()
clf.fit(X_train,train_labels2)
#clf.score(X_test, y_test)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [533]:
clf.score(X_test,test_labels)

0.7192259869443581

In [224]:
y_pred_svm = clf.predict(X_test)

In [225]:
confusion_matrix(y_true,y_pred_svm)
#np.unique(train_labels2 == typeLst)

array([[603,  51,  73,  23,  50,  27, 106,  33,  34,   0],
       [101, 614,  37,  15,  13,  24,  97,  38,  61,   0],
       [117,  50, 578,  40,  34,  42,  98,   5,  34,   2],
       [ 17,  10,  25, 890,   5,  22,  19,   5,   7,   0],
       [ 81,  24,  31,  15, 382,   8,  54,  16,  11,   1],
       [ 31,  34,  45,  17,   5, 824,  12,   7,  24,   1],
       [173,  92,  72,  28,  27,  18, 501,  31,  57,   1],
       [ 42,  37,  13,   4,   8,   7,  33, 829,  27,   0],
       [ 58,  46,  25,  25,  11,  24,  41,  12, 755,   3],
       [ 21,  47,   8,   3,   6,   8,  21,   7,   9, 870]])

In [534]:
#Decicion tree
from sklearn import tree

treeClf = tree.DecisionTreeClassifier()
treeClf.fit(X_train, train_labels2)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [543]:
treeClf.score(X_test, test_labels)

0.2661641280696301

In [566]:
from sklearn.ensemble import RandomForestClassifier
forestClf = RandomForestClassifier()
forestClf.fit(X_train, train_labels2)

ValueError: Number of labels=38489 does not match number of samples=51466

In [542]:
forestClf.score(X_test, test_labels)

0.35708734846129936

In [203]:
from sklearn.neighbors import KNeighborsClassifier
KNN_clf = KNeighborsClassifier(n_neighbors=5)
KNN_clf.fit(X_train, train_labels2)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [207]:
KNN_clf.score(X_test, test_labels)

0.5564792684194119

In [605]:
np.unique(y_pred1)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 11.], dtype=float32)

In [593]:
confusion_matrix(y_true,y_pred1)

array([[252, 131, 121,  46, 244,  63, 101,  28,  12,   2],
       [ 27, 504,  69,  19,  99,  87, 103,  67,  23,   2],
       [ 54, 102, 408,  54, 186,  81,  79,  15,  21,   0],
       [ 30,  89, 150, 266, 181, 124,  92,  19,  44,   5],
       [ 11,  50,  63,  13, 422,  21,  25,  13,   5,   0],
       [  8,  55,  68,  20, 130, 691,  18,   8,   2,   0],
       [ 40, 186,  94,  38, 182,  46, 363,  32,  18,   1],
       [ 25,  79,  13,  10, 178,  54,  35, 591,  15,   0],
       [ 16, 119,  50,  67,  83,  63,  78,  43, 474,   7],
       [ 11,  46,  14,   9, 167,  15,  22,   6,   3, 707]])

In [389]:
#try tf-idf svm
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(train["content"])

In [609]:
# Setup scikit
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, log_loss
from sklearn.pipeline import Pipeline

# models
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier


# Naive Bayes
clf_NB = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   ('NB', MultinomialNB()),])
                   
# Support vector machine
clf_svm = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('svm', SGDClassifier()),])

In [644]:
clf_svm.fit(series_content,typeLst )

In [631]:
np.unique(test_labels == test_typeLst, return_counts = True)

(array([ True]), array([9623]))

In [646]:
clf_svm.score(test_content, test_labels)

0.7225397485191728

In [645]:
clf_svm.score(test_content, test_labels)clf_svm.score(test_content, test_labels)

SyntaxError: invalid syntax (<ipython-input-645-bc1c4179375a>, line 1)

In [585]:
#X_train
#len(train_labels2)

51466

In [589]:
train_labels2 = np.array(train_labels2)
len(train_labels2)

51466

In [581]:
#(X_train)
#len(train_labels2)

In [83]:
import tensorflow as tf
# laver modellen
model = tf.keras.models.Sequential()
# tilføjer et input på modellen
model.add(tf.keras.layers.Flatten())

# relu er default aktiverings funktion. Lav den om hvis resultatet ikke er godt nok
model.add(tf.keras.layers.Dense(6, activation=tf.nn.relu))
# jeg tilføjer 2 lag til netwærket. Dette er fordi det er en simpel opgave
model.add(tf.keras.layers.Dense(6, activation=tf.nn.relu))

#antallet (10) er antal output. Det er 10 tal i datasettet derfor skal der være et 10 tal
model.add(tf.keras.layers.Dense(12, activation=tf.nn.softmax))

# Dette er den mest komplexe del. adam er goto. Hvis der kun er 2 løsninger så brug binary_categorical_crossentropy eller binary_crossentropy i stedet for sparse_categorical_crossentropy
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [89]:
#(X_train[0])

In [84]:
model.fit(train_arrays2, train_labels2, epochs = 3)

Epoch 1/3
1609/1609 [==============================] - 5s 3ms/step - loss: 1.8646 - accuracy: 0.3420
Epoch 2/3
1609/1609 [==============================] - 4s 3ms/step - loss: 1.3896 - accuracy: 0.5305
Epoch 3/3
1609/1609 [==============================] - 5s 3ms/step - loss: 1.2958 - accuracy: 0.5623


In [101]:
from nltk.stem import PorterStemmer 
mySent = "These are my sentences in the books"
mySentTok = word_tokenize(mySent)
ps = PorterStemmer()
for w in mySentTok:
    mySentTok[w] = ps.stem(w)

TypeError: list indices must be integers or slices, not str